# <span style='color:#547DCD'> Portfolio backtesting  </span> 

## <span style='color:#7F8BC7'> Setting the protocol  </span> 

Consider a dataset with three dimensions: 
- Time: $t = 1, \ldots , T$,
- Assets: $n = 1, \ldots , N$,
- Characteristics: $k = 1, \ldots , K$ (one of these attributes must be price of asset $n$ at time $t$, denoted $p_{t,n}$).

Arithmetic return is 
$$
 r_{t,n} = \frac{p_{t,n}}{p_{t-1,n}} - 1
$$

Assume that points in time are equidistant or uniform. If each point in time $t$ has data available for all assets then we have a dataset with $I = T \times N$ rows.

First split the dataset in two: out of sample period and initial buffer period, which is required to train the the models for the first portfolio composition. The size of the buffer period is determined by size of training sample. The size of the training sample can be chosen two ways: fixed (2 to 10 years) or expanding. 
- Fixed: training sample will roll over time, taking into account only the most recent data.
- Expanding: Models are built on all the available data (size of which increases with time).
    - Can create problems since the first dates of the backtests are based on much smaller amounts of information compared to the last dates.
    - In addition, there is an ongoing debate as to whether or not using the full history of returns is advantageous or not (pros: models see different markets, cons: old data is outdated adn useless, possibly misleading).

Going forward, the fixed rolling period for the training sample. 

Two crucial design choices are the **rebalancing frequency** and the **horizon** at which the label is computed. It is not obvious that they should be equal but their choice should make sense.

These choices have a direct impact on how the backtest is carried out. If we note:
- $\Delta_{h}$ for the holding period between 2 rebalancing dates (in days or months);
- $\Delta_s$ for the size of the desired training sample (not taking the number of assets into consideration); 
- $\Delta_l$ for the horizon at which the label is computed.

then total length of the training sample should be $\Delta_s + \Delta_l$. INdeed at any moment $t$, the training sample should stop at $t - \Delta_l$, so the last point corresponds to a label that i calculated at time $t$. 

To avoid **forward-looking bias**, it’s crucial that the training data for a predictive model only includes information available up to the point of prediction, not future data. The training sample should end at $t - \Delta_l$, where $t$ is the current time and $\Delta_l$ is the label horizon. Any data used to predict the outcome should not extend into the future beyond this point. The "red zone" refers to the period from $(t - \Delta_l$ $t$, and using data within this range leads to forward-looking bias, as it involves future information that wouldn't realistically be available when making the prediction. This bias can artificially inflate a model’s performance during backtesting, making it seem more accurate than it actually is.

Consider a observation with index $s$ inside the interval $(t - \Delta_l, t]$ will entail a forward looking bias. If a feature is indexed by $s \in (t - \Delta_l, t]$, then the label will by design cover $[s, s + \Delta_l]$ where $s + \Delta_l > t$, which mean that at time $t$ it requires knowledge of the future, which is not realistic.


## <span style='color:#7F8BC7'> Turning signals into portfolio weights  </span> 

Previous predictive tools are meant to provide a signal that is expected to give information on the future profitability of assets; these signals can be integrated in an investment decision. 

There are two steps (signal can be used on at any of these stages). Relying on the signal for both steps puts a lot of emphasis on the predictions and should only be considered when the level of confidence in the forecasts is high.

**First step (Selection**): We do not need to invest in all assets investigated of them. *For long-only portfolios, it makes sense to take advantage of the signal toexclude those assets, that are likely to underperform in the future. Often, portfolio policies have fixed sizes that impose a constant number of assets. A heuristic way of exploiting the signal is to select the assets with the best predictions and discard the others. Portfolios can be constructed using the quantiles of underlying characteristics and some characteristics are deemed interesting if the corresponding sorted portfolios exhibit different profitabilities (e.g., high average returns for high quantiles versus low average return for low quantiles).*

This is efficient in testing the relevance of the signal. If $Q$ portfolios $q = 1, \ldots , Q$ are formed according to the rankings of the assets with respect to the signal, then one would expect that the out-of-sample performance of the portfolios be monotonic with $q$. We often assume that the extreme portfolios suffice; if the difference between portfolio 1 and $Q$ is substantial, then the signal is valuable.'

**Second step (weighting)**: When selection is done on signal, then simple weighting is a good idea. Equal weight portfolios are hard to beat. More advanced schemes include equal risk contributions and constrained minimum variance; both rely on the covariance matrix of the assets.

### <span style='color:#AA9AC2'> Generalization of Coqueret  </span> 

Consider the generic constrained quadratic programme
$$
\min_{\mathbf{w}} \frac{\lambda}{2} \mathbf{w}'  \boldsymbol{\Sigma} \mathbf{w} - \mathbf{w}' \mathbf{\mu}, \quad \text{s.t.} \quad \begin{matrix}
\mathbf{w}' \mathbf{1} = 1 \\
(\mathbf{w} - \mathbf{w}_{-})' \mathbf{\Lambda} (\mathbf{w} - \mathbf{w}_{-}) \le \delta_{R}, \\
\mathbf{w}'\mathbf{w} \le \delta_{D}
\end{matrix}
$$

The first constraint is the budget constraint (weights som to one). The second is a penalization constraint for variations in weights (compared to current allocation $\mathbf{w}_{-}$) via the diagonal matrix $\mathbf{\Lambda}$ that penalizes trading costs. $\mathbf{w} - \mathbf{w}_{-}$ is the vector of deviations from current position, and $(\mathbf{w} - \mathbf{w}_{-})' \mathbf{\Lambda} (\mathbf{w} - \mathbf{w}_{-})$ is an expression that characterizes the sum of squared deviations, weighted by the the diagonal coefficients $\Lambda_{n,n}$. This can be helpful because some assets may be more costly to trade due to liquidity. When $\delta_{R}$ decreases, the rotation is reduced. The last constraint enforces diversification via the Herfindhal-Hirschmann index of the portfolio: the smaller the $\delta_{D}$,the more diversified the portfolio.

Recall that there are $N$ assets, then the Langrange form of the above becomes: 
$$
L(\mathbf{w}) = \frac{\lambda}{2} \mathbf{w}' \boldsymbol{\Sigma} \mathbf{w} - \mathbf{w}' \boldsymbol{\mu} - \eta (\mathbf{w}' \mathbf{1}_{N} - 1) + \kappa_{R}((\mathbf{w} - \mathbf{w}_{-})' \boldsymbol{\Lambda} (\mathbf{w} - \mathbf{w}_{-}) - \delta_{R}) + \kappa_{D}(\mathbf{w}'\mathbf{w} - \delta_{D})
$$

and the first order condition 
$$
\frac{\partial }{\partial \mathbf{w}} L(\mathbf{w}) = \lambda \mathbf{\Sigma} \mathbf{w} - \mu - \eta \mathbf{1}_N + 2 \kappa_R \mathbf{\Lambda} (\mathbf{w} - \mathbf{w}_-) + 2 \kappa_D \mathbf{w} = 0,
$$
yields
$$
\mathbf{w}_{\kappa}^{*} = \left( \lambda \mathbf{\Sigma} + 2 \kappa_R \mathbf{\Lambda} + 2 \kappa_D \mathbf{I}_N \right)^{-1} \left( \mathbf{\mu} + \eta_{\lambda, \kappa_R, \kappa_D} \mathbf{1}_N + 2 \kappa_R \mathbf{\Lambda} \mathbf{w}_- \right),
$$
with
$$
\eta_{\lambda, \kappa_R, \kappa_D} = \frac{1 - \mathbf{1}_N' (\lambda \mathbf{\Sigma} + 2 \kappa_R \mathbf{\Lambda} + 2 \kappa_D \mathbf{I}_N)^{-1} (\mathbf{\mu} + 2 \kappa_R \mathbf{\Lambda} \mathbf{w}_-)}
{\mathbf{1}_N' (\lambda \mathbf{\Sigma} + 2 \kappa_R \mathbf{\Lambda} + 2 \kappa_D \mathbf{I}_N)^{-1} \mathbf{1}_N}.
$$


This parameter ensures that the budget constraint is satisfied. Optimal weights depend on the tuning parameters: $\lambda$, $\kappa_{R}$ and $\kappa_{D}$.
- When $\lambda$ is large, the focus is set more on risk reduction than on profit maximization (which is often a good idea given that risk is easier to predict).
- When $\kappa_{R}$ is large, the importance of transaction costs in (12.2) is high and thus, in the limit when $\kappa_{R} \to \infty$, the optimal weights are equal to the old ones $\mathbf{w}_{-}$.
- When $\kappa_{R}$ is large, the portfolio is more diversified and (all other things equal) when $\kappa_D \to \infty$  the weights are equal to $\frac{1}{N}$.
- When $\kappa_{D} = \kappa_{R} = 0$, we recover the mean-variance weights.

See exercise for tests and adjustments for choosing realistic values of the tuning parameters.